In [ ]:
from google.colab import drive
import os
import ee
import time

# ---------------------------
# 1. Mount Google Drive
# ---------------------------
drive.mount('/content/drive', force_remount=True)

# Base export folder
export_folder = '/content/drive/My Drive/GEE_Exports/'
dataset_name = "FIRMS"
epsm = 1000  # 1000m resolution
attributes = ['T21', 'confidence']

# Define province and year variables
country = "Canada"
province = "Alberta"
year = "2023"

# List of months to process with their start and end dates
months_to_run = {
    "07": (f'{year}-07-01', f'{year}-07-31'),
    "08": (f'{year}-08-01', f'{year}-08-31'),
    "09": (f'{year}-09-01', f'{year}-09-30'),
}

# ---------------------------
# 2. Initialize Earth Engine
# ---------------------------
cloud_project = 'ee-1wangjas2'

try:
    ee.Initialize(project=cloud_project)
    print('Earth Engine initialized successfully.')
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)
    print('Earth Engine initialized after authentication.')

# Load level 2 administrative boundaries
gaul_cities = ee.FeatureCollection('FAO/GAUL/2015/level2')
canada_cities = gaul_cities.filter(ee.Filter.eq('ADM0_NAME', country))
province_cities = canada_cities.filter(ee.Filter.eq('ADM1_NAME', province))

# ---------------------------
# Helper Functions
# ---------------------------
def estimate_computation_complexity(geometry, scale):
    area = geometry.area(1).getInfo()
    grid_count = area / (scale * scale)
    return grid_count

def get_optimal_tile_scale(grid_count):
    if grid_count > 1000000:
        return 2
    elif grid_count > 100000:
        return 4
    elif grid_count > 10000:
        return 8
    else:
        return 16

def zonal_stats_with_date(feature_collection, image, tile_scale):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=epsm,
        tileScale=tile_scale
    )
    return stats.map(lambda f: f.set({
        'longitude': f.geometry().centroid(1).coordinates().get(0),
        'latitude': f.geometry().centroid(1).coordinates().get(1),
        'date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
        **{band: f.get(band) for band in attributes}
    }))

def process_city(feature, start_date, end_date):
    city_name = feature.get('ADM2_NAME').getInfo().replace(" ", "_")
    region_geometry = feature.geometry()

    grid_count = estimate_computation_complexity(region_geometry, epsm)
    tile_scale = get_optimal_tile_scale(grid_count)

    firms_filtered_city = (ee.ImageCollection(dataset_name)
                           .filterDate(start_date, end_date)
                           .filterBounds(region_geometry)
                           .select(attributes))

    projection = ee.Projection('EPSG:4326').atScale(epsm)
    grids = region_geometry.coveringGrid(projection)

    per_image_stats = firms_filtered_city.map(lambda image: zonal_stats_with_date(grids, image, tile_scale))
    all_stats = ee.FeatureCollection(per_image_stats.flatten())

    # Export to Google Drive
    export_task = ee.batch.Export.table.toDrive(
        collection=all_stats,
        description=f'{city_name}_{dataset_name}_{start_date}',
        folder=f'5000_epsm_{dataset_name}/{start_date[:7]}',
        fileNamePrefix=f'{city_name}_{dataset_name}_{start_date}',
        fileFormat='CSV',
        selectors=['date', 'longitude', 'latitude'] + attributes
    )
    export_task.start()
    print(f"Export task for {city_name} started with tileScale {tile_scale}")

# ---------------------------
# Main Processing Loop
# ---------------------------
for month, (start_date, end_date) in months_to_run.items():
    province_folder = os.path.join(export_folder, province)
    if not os.path.exists(province_folder):
        os.makedirs(province_folder)

    year_folder = os.path.join(province_folder, year)
    if not os.path.exists(year_folder):
        os.makedirs(year_folder)

    month_folder = os.path.join(year_folder, month)
    if not os.path.exists(month_folder):
        os.makedirs(month_folder)

    dataset_folder = os.path.join(month_folder, dataset_name)
    if not os.path.exists(dataset_folder):
        os.makedirs(dataset_folder)

    os.chdir(dataset_folder)
    print(f'Current Directory: {os.getcwd()}')

    province_city_list = province_cities.toList(province_cities.size())
    city_count = province_cities.size().getInfo()
    print(f"Processing {city_count} cities in {province} for {month}-{year}")

    for i in range(city_count):
        city_feature = ee.Feature(province_city_list.get(i))
        process_city(city_feature, start_date, end_date)

print("All export tasks initiated. Check the Earth Engine Tasks tab for progress.")


Mounted at /content/drive
Earth Engine initialized after authentication.
Current Directory: /content/drive/My Drive/GEE_Exports/Alberta/2023/07/FIRMS
Processing 19 cities in Alberta for 07-2023
Export task for Division_No.__1 started with tileScale 8
Export task for Division_No.__2 started with tileScale 8
Export task for Division_No.__3 started with tileScale 8
Export task for Division_No.__4 started with tileScale 8
Export task for Division_No.__5 started with tileScale 8
Export task for Division_No.__6 started with tileScale 8
Export task for Division_No.__7 started with tileScale 8
Export task for Division_No.__8 started with tileScale 8
Export task for Division_No.__9 started with tileScale 8
Export task for Division_No._10 started with tileScale 8
Export task for Division_No._11 started with tileScale 8


KeyboardInterrupt: 